In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt
from pandas import Series
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import gmplot

#importing the dataset using pandas
df = pd.read_csv("./911.csv")

#sample of original dataset
df.head(5)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#separate timeStamp data into 2 new columns
df['date'] = df.timeStamp.str[0:11]
df['time'] = df.timeStamp.str[-8:]

#Get rid of dummy 'e' column and 'timeStamp' column
del df['e']
del df['timeStamp']
#If time at end, then try to extract station number and impute
del df['desc']


In [ ]:
#count number of values in each column
df.count()

In [ ]:
#Finds total number of rows with missing values 
#Rows with more than one missing value only counted once
df.isnull().any(axis=1).sum()

In [ ]:
#Drop rows with missing zipcode values
df = df.dropna(subset=['zip'])

#Convert float values for zipcodes to integer type
df['zip'] = df['zip'].astype(int)
df.count()

In [ ]:
empty = np.where(pd.isnull(df))
geolocator = Nominatim()
index = 0


#Impute 24 missing township values
for i in np.nditer(empty):
    
    #row of missing township cell
    row = empty[0][index] 
    #column of missing township cell
    column = empty[1][index]
    
    
    temp_lat = repr(df.iloc[row,0])
    temp_long = repr(df.iloc[row,1])
    
    
    location = geolocator.reverse([temp_lat, temp_long], timeout = 60)
    
    if column == 4:
        
    
        #extract township value from location dictionary
        town = location.raw['address']['city'] 
    
    
        #remove 'Township' ending from name of town    
        if town.endswith("Township"):
            town = town[0:-9]
            
        else:
            pass
                
        #convert to uppercase to maintain township format in dataframe    
        town = town.upper()
    
        #put imputed township name into corresponding missing cell of dataframe
        df.iloc[row, column] = town
    
        print(df.iloc[row, column])
        
    else:
        
        pass
        
        #Elected to comment out code to impute missing zipcodes because it would take too long (approx. 4-6 hrs.)
        #zcode = location.raw['address']['postcode']
        
        #df.iloc[row, column] = zcode
        
        #print(df.iloc[row, column])
    
        
    #increment index to get to next set of index values for empty township cell
    index += 1

In [ ]:
#After imputing townships: No more missing values in the dataset
df.count()

In [ ]:
hour = df.time.str[0:2]
hour2 = pd.to_numeric(hour)

#If time of call is between 6PM and 6AM then it is classified as 'night', otherwise it is classified as 'day'

for i, row in df.iterrows():
    if(hour2.loc[i] >= 18 or hour2.loc[i] < 6):
        hour.at[i] = 'night'
    else:
        hour.at[i] = 'day'


In [ ]:
#Replace military time with either 'night' or 'day'
del df['time']

df['time_of_day'] = hour   

In [ ]:
#Change date format to weekdays format (ex. Monday, Tuesday ...)
df['dates'] = pd.to_datetime(df['date'])
df['weekday'] = df['dates'].dt.weekday_name

del df['date']
del df['dates']

In [ ]:
#Separate first part of 911 call classification from rest of title
df['class'], df['title2'] = df['title'].str.split(':', 1).str
del df['title']
del df['title2']


In [ ]:
df.head(10)

In [ ]:
#end of general preprocessing

In [ ]:
#unique zipcodes in the dataset
df.zip.unique()

In [ ]:
#find the number of unique zipcodes
s = Series(df.zip)
zip_unique = s.unique().size
print(zip_unique)

In [ ]:
#unique townships in the dataset
df.twp.unique()

In [ ]:
#find the number of unique townships
s2 = Series(df.twp)
twp_unique = s2.unique().size
print(twp_unique)

In [ ]:
#find the number of unique address locations
s4 = Series(df.addr)
s4.unique().size

In [ ]:
#find number of unique latitude values
sLat = Series(df.lat)
sLat.unique().size

In [ ]:
#find number of unique longitude values
sLong = Series(df.lng)
sLong.unique().size

In [ ]:
#find max, min latitudes and longitudes
df.describe()

In [ ]:
# used code from https://www.kaggle.com/vishnoiprem/d/mchirico/montcoalert/911-calls-visualization
# modified some code and added to the visulization
DATA = np.zeros((df.shape[0],7),dtype='O')
DATA[:,0] = df['lng'].values
DATA[:,1] = df['lat'].values
DATA[:,2] = df['zip'].values
DATA[:,3] = df['weekday'].values
DATA[:,4] = df['time_of_day'].values
DATA[:,5] = df['addr'].values
DATA[:,6] = df['class'].values


In [ ]:
typeOfCall = np.zeros(DATA.shape[0],dtype='O')
for i in range(typeOfCall.size):
    typeOfCall[i] = DATA[i][6]

In [ ]:
call_type = np.array(["Ems", "Fire", "Traffic"])
sns.plt.figure(figsize=(12,4))
sns.plt.xlabel("Type of Incident")
sns.plt.title("All Situations By Time")
sns.countplot(typeOfCall)

In [ ]:
#Additional Preprocessing for Bagging and Random Forest

In [ ]:
#Change string data to label encoded integers for Bagging & RandomForest Classifier
le_dow = preprocessing.LabelEncoder()
le_tod = preprocessing.LabelEncoder()
le_addr = preprocessing.LabelEncoder()
le_twp = preprocessing.LabelEncoder()
le_class = preprocessing.LabelEncoder()

le_dow = le_dow.fit_transform(df['weekday'])
le_tod = le_tod.fit_transform(df['time_of_day'])
le_addr = le_addr.fit_transform(df['addr'])
le_twp = le_twp.fit_transform(df['twp'])
le_class = le_class.fit_transform(df['class'])

In [ ]:
#Create copy of dataframe
df2 = df.copy()

#Delete old columns with string values and replace with new label encoded columns
del df2['weekday']
del df2['time_of_day']
del df2['addr']
del df2['twp']
del df2['class']


df2['weekday'] = le_dow
df2['time_of_day'] = le_tod
df2['addr'] = le_addr
df2['twp'] = le_twp
df2['class'] = le_class

df2.head(10)

In [ ]:
#split dataset for building Bagging classifier
x = df2[['lat', 'lng', 'zip', 'twp', 'addr', 'time_of_day', 'weekday']].copy()
y = df2[['class']].copy()
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)

In [ ]:
#view input attributes for Bagging and Random Forest classifiers
x.head()

In [ ]:
#view output values for Bagging and Random Forest
y.head()

In [ ]:
bag = BaggingClassifier(base_estimator=None, max_features=7, n_estimators=100, n_jobs=-1, random_state=0)
bag.fit(x_train, y_train.values.ravel())

In [ ]:
print("Accuracy on training set: {:.3f}".format(bag.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(bag.score(x_test, y_test)))

In [ ]:
bag1 = BaggingClassifier(base_estimator=KNeighborsClassifier(n_jobs=-1, p=1, n_neighbors=10), max_features=7, n_estimators=100, n_jobs=-1, random_state=0)
bag1.fit(x_train, y_train.values.ravel())

In [ ]:
print("Accuracy on training set: {:.3f}".format(bag1.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(bag1.score(x_test, y_test)))

In [ ]:
#Random Forest Classification

In [ ]:
#Manual 80/20 Train/Test split since data is timeseries data (newer data should be test data)
x_train2 = x[:124766]
x_test2 = x[124766:]

y_train2 = y[:124766]
y_test2 = y[124766:]

forest2 = RandomForestClassifier(n_estimators=100, criterion='entropy', max_features=5, max_depth=25, 
                                 min_impurity_split=0.9, n_jobs=-1, random_state=0)

forest2.fit(x_train2, y_train2.values.ravel())

In [ ]:
print("Accuracy on training set: {:.3f}".format(forest2.score(x_train2, y_train2)))
print("Accuracy on test set: {:.3f}".format(forest2.score(x_test2, y_test2)))

In [ ]:
important = forest2.feature_importances_
print ('Feature importance:\n')

#Print importance value for each feature
for i in range(7):
    print(df2.columns.values[i],': ',important[i], '\n')
    

In [ ]:
n_features = 7
names = []

for i in range(n_features):
    names.append(df2.columns.values[i])

#Plot importance of attributes for Random Forest Classifier
plt.barh(range(n_features), forest2.feature_importances_, align='center')
plt.yticks(np.arange(n_features), names)
plt.xlabel("911 Dataset Feature importance")

In [ ]:
#SVC (Support Vector Classification)

In [ ]:
#make a copy of data set to start transforming attributes into float 
df3= df.copy()

df3.head()

#uncomment the next two lines to encode the addresses for input in classification
le_addr = preprocessing.LabelEncoder()
le_addr = le_addr.fit_transform(df3['addr'])


#delete addrs attribute
#del df3['addr']

#uncomment next line and comment out previous line if keeping addresses is desired
df3['addr'] = le_addr


In [ ]:
#transformed dataframe to dictionary for additional preprocessing
comb = [df3]

In [ ]:
for dataset in comb:
    dataset['time_of_day'] = dataset['time_of_day'].map( {'day': 1,'night': 2} ).astype(float)

In [ ]:
for dataset in comb:
    dataset['class'] = dataset['class'].map( {'Traffic': 10,'EMS': 20, 'Fire': 30} ).astype(float)

In [ ]:
for dataset in comb:
     dataset['weekday'] = dataset['weekday'].map( {'Monday': 11,'Tuesday': 22,'Wednesday': 33,'Thursday': 44,'Friday': 55,'Saturday': 66, 'Sunday': 77} ).astype(float) 

In [ ]:
#Using unique float values for each mapping
for dataset in comb:
    dataset['twp'] = dataset['twp'].map({'NEW HANOVER' :101, 'HATFIELD TOWNSHIP': 111, 'NORRISTOWN': 12, 'LANSDALE': 13,
       'HORSHAM': 14, 'SKIPPACK': 15, 'LOWER SALFORD': 16, 'PLYMOUTH': 17,
       'UPPER MORELAND': 18, 'CHELTENHAM': 19, 'MONTGOMERY': 202, 'WHITEMARSH': 21,
       'UPPER GWYNEDD': 222, 'LOWER PROVIDENCE': 23, 'WHITPAIN': 24, 'DELAWARE COUNTY': 25,
       'FRANCONIA': 76, 'WEST CONSHOHOCKEN': 777, 'UPPER MERION': 78, 'LIMERICK': 79,
       'DOUGLASS': 26, 'LOWER MERION': 27, 'POTTSTOWN': 28, 'BRIDGEPORT': 29, 'TOWAMENCIN': 303,
       'AMBLER': 31, 'LOWER POTTSGROVE': 32, 'CHESTER COUNTY': 333, 'UPPER HANOVER': 34,
       'SPRINGFIELD': 35, 'ROCKLEDGE': 36, 'ABINGTON': 37, 'WEST NORRITON': 38,
       'ROYERSFORD': 39, 'UPPER DUBLIN': 40, 'UPPER SALFORD': 41, 'CONSHOHOCKEN': 42,
       'PENNSBURG': 43, 'TELFORD': 444, 'EAST NORRITON': 45, 'UPPER FREDERICK': 46,
       'UPPER PROVIDENCE': 47, 'SALFORD': 48, 'LEHIGH COUNTY': 49, 'MARLBOROUGH': 50,
       'BRYN ATHYN': 51, 'LOWER MORELAND': 52, 'HATBORO': 53, 'LOWER GWYNEDD': 54,
       'WORCESTER': 555, 'COLLEGEVILLE': 56, 'SCHWENKSVILLE': 57, 'SOUDERTON': 58,
       'PERKIOMEN': 59, 'LOWER FREDERICK': 60, 'BUCKS COUNTY': 61, 'RED HILL': 62,
       'WEST POTTSGROVE': 63, 'UPPER POTTSGROVE': 64, 'EAST GREENVILLE': 65,
       'NORTH WALES': 666,'JENKINTOWN': 67,'TRAPPE': 68, 'NARBERTH': 69, 'BERKS COUNTY': 70,
       'GREEN LANE': 71, 'WARRINGTON': 72, 'PHILA COUNTY': 73, 'HATFIELD': 74,
       'HATFIELD BORO': 75}).astype(float)

In [ ]:
df3.head(5)

In [ ]:
#Splitting data into training and testing sets
#uncoment this next line and comment out the following line to keep address feature
x3 = df3[['lat', 'lng', 'zip', 'addr', 'twp', 'time_of_day', 'weekday']].copy()

#x3 = df3[['lat', 'lng', 'zip', 'twp', 'time_of_day', 'weekday']].copy()
y3 = df3[['class']].copy()

In [ ]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(x3, y3, stratify=y, random_state=42)

In [ ]:
#Create Linear SVC model
model = OneVsRestClassifier(LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, 
                                      class_weight=None, verbose=0, random_state=None, max_iter=100))

model.fit(x_train3,y_train3)

In [ ]:
print("Accuracy on training set: {:.3f}".format(model.score(x_train3, y_train3)))
print("Accuracy on test set: {:.3f}".format(model.score(x_test3, y_test3)))

In [ ]:
#uncomment next two blocks of code to use kernel SVC, runtime is approximately 1-1.5 hours
#from sklearn import svm

clf = svm.SVC()
clf.fit(x_train3,y_train3.values.ravel())

In [ ]:
print("Accuracy on training set: {:.3f}".format(clf.score(x_train3, y_train3)))
print("Accuracy on test set: {:.3f}".format(clf.score(x_test3, y_test3)))

In [ ]:
#Arrays for all latitudes and longitudes 
lats = []
longs = []

#Arrays for latitude and longitude of each different 911 call type
ems_lat = []
ems_long = []
fire_lat = []
fire_long = []
traffic_lat = []
traffic_long = []


#Put latitute and longitude of 911 calls into arrays for maps
for i in range(155957):
    lats.append(df2.iloc[i, 0])
    longs.append(df2.iloc[i, 1])
    
    #911 call for EMS
    if df2.iloc[i, 7] == 0:
        ems_lat.append(df2.iloc[i, 0])
        ems_long.append(df2.iloc[i, 1])
    
    #911 call for Fire
    elif df2.iloc[i, 7] == 1:
        fire_lat.append(df2.iloc[i, 0])
        fire_long.append(df2.iloc[i, 1])
    
    #911 call for Traffic
    else:
        traffic_lat.append(df2.iloc[i, 0])
        traffic_long.append(df2.iloc[i, 1])

In [ ]:
#Latitudes and Longitudes for plotting boundaries of Montgomery County Pennsylvania
lat_bound = (40.241979, 40.447123, 40.138185, 40.069056, 40.063658, 40.046278, 40.084713, 
40.073154, 40.092990, 40.054093, 40.011573, 39.977010, 40.019547, 40.015951, 40.072212, 
40.060884, 40.097123, 40.090095, 40.094000, 40.087753, 40.115236, 40.126164, 40.129910,
40.147977, 40.194150, 40.223927, 40.236395, 40.241979)

long_bound = (-75.696874, -75.529820, -75.015042, -75.096642, -75.087832, -75.110155,
-75.176633, -75.188473, -75.223789, -75.264413, -75.206578, -75.276598, -75.311506, 
-75.320488, -75.367236, -75.392806, -75.420365, -75.437921, -75.440779, -75.456498, 
-75.471208, -75.463654, -75.500604, -75.524058, -75.569582, -75.608164, -75.687167, -75.696874)                    


#Create map centered at lat, long, amount of zoom
gmap = gmplot.GoogleMapPlotter(40.194126, -75.362524, 8.5)

#Plot outline of Montgomery County
gmap.plot(lat_bound, long_bound, 'cornflowerblue', edge_width=5)

#Plot 911 call location in heatmap form
gmap.heatmap(lats, longs)

#Save html file of Google map
gmap.draw("Montgomery_County_Heatmap.html")

In [ ]:
#Create map centered at lat, long, amount of zoom
gmap2 = gmplot.GoogleMapPlotter(40.194126, -75.362524, 10)

#Plot outline of Montgomery County
gmap2.plot(lat_bound, long_bound, 'cornflowerblue', edge_width=5)

#Plot first 10,000 EMS, Fire, and Traffic 911 call locations onto map (30,000 call locations total)
#EMS = green
gmap2.scatter(ems_lat[0:10000],ems_long[0:10000] , 'g', 5, marker=False)
#Fire = red
gmap2.scatter(fire_lat[0:10000],fire_long[0:10000] , 'r', 5, marker=False)
#Traffic = yellow
gmap2.scatter(traffic_lat[0:10000],traffic_long[0:10000] , 'y', 5, marker=False)


#Save html file of Google map
gmap2.draw("911_Call_Locations.html")